In [1]:
!pip install -q timm pytorch-metric-learning

In [2]:
import os
import cv2
import copy
import time
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.cuda import amp
from torchvision import datasets, models, transforms

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score, f1_score

from tqdm.notebook import tqdm
from collections import defaultdict
import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm
from pytorch_metric_learning import losses

Adding direction path for my locel device

In [3]:
dataset_path=r'C:\Users\eddyl_kayxd9j\OneDrive\Desktop\VGG-19\Selfsupervisd learning'

In [4]:
class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b4_ns'
    img_size = 224
    scheduler = 'CosineAnnealingLR'
    T_max = 10
    lr = 1e-4 #1e-4
    min_lr = 1e-6 #1e-6
    batch_size = 8 #16
    weight_decay = 1e-6
    num_epochs = 10
    num_classes = 2 # 2 output classes for my label (0 and 1)
    embedding_size = 224
    n_fold = 2
    n_accumulate = 4
    temperature = 0.1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:

TRAIN_DIR = r'C:\Psychopy\Experiment\Cat experiment\cat dataset'
TEST_DIR = r'C:\Psychopy\Experiment\Cat experiment\cat dataset'+'/test_images/'

In [6]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    torch.cuda.empty_cache() 
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    

set_seed(CFG.seed)

Loading my label data

In [7]:
df_train = pd.read_csv(path+'/random_sample_test.csv')

In [ ]:
# le = LabelEncoder()
# df_train.label= le.fit_transform(df_train.label)

Dataset Class

In [8]:
class ShopeeDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.df.iloc[index, 0]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[index, 1]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return img, label

Augmentations & Transforms

In [9]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

Training function

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs, dataloaders, dataset_sizes, device, fold):
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    scaler = amp.GradScaler()

    for step, epoch in enumerate(range(1,num_epochs+1)):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','valid']:
            if(phase == 'train'):
                model.train() # Set model to training mode
            else:
                model.eval() # Set model to evaluation mode
            
            running_loss = 0.0
            
            # Iterate over data
            for inputs,labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(CFG.device)
                labels = labels.to(CFG.device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    with amp.autocast(enabled=True):
                        print(torch.any(torch.isnan(inputs)))
                        outputs = model(inputs)
                        print('ouputs_shape:{}'.format(outputs.shape))
                        print('ouputs:{}'.format(outputs))
                        print('outputs_len:{}'.format(len(outputs)))
                        print('labels:{}'.format(labels))
                        loss = criterion(outputs, labels)
                        loss = loss / CFG.n_accumulate
                    
                    # backward only if in training phase
                    if phase == 'train':
                        scaler.scale(loss).backward()

                    # optimize only if in training phase
                    if phase == 'train' and (step + 1) % CFG.n_accumulate == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        scheduler.step()
                        
                        # zero the parameter gradients
                        optimizer.zero_grad()


                running_loss += loss.item()*inputs.size(0)
            
            epoch_loss = running_loss/dataset_sizes[phase]            
            history[phase + ' loss'].append(epoch_loss)
            print(loss)
            print(running_loss)
            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))
            
            # deep copy the model
            if phase=='valid' and epoch_loss <= best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                PATH = f"Fold{fold}_{best_loss}_epoch_{epoch}.bin"
                torch.save(model.state_dict(), PATH)

        print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss ",best_loss)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [11]:
def run_fold(model, criterion, optimizer, scheduler, device, fold, num_epochs=10):
    valid_df = df_train[df_train.fold == fold]
    train_df = df_train[df_train.fold != fold]
    
    train_data = ShopeeDataset(TRAIN_DIR, train_df, transforms=data_transforms["train"])
    valid_data = ShopeeDataset(TRAIN_DIR, valid_df, transforms=data_transforms["valid"])
    
    dataset_sizes = {
        'train' : len(train_data),
        'valid' : len(valid_data)
    }
    
    train_loader = DataLoader(dataset=train_data, batch_size=CFG.batch_size, num_workers=0, pin_memory=True, shuffle=True)
    valid_loader = DataLoader(dataset=valid_data, batch_size=CFG.batch_size, num_workers=0, pin_memory=True, shuffle=False)
    
    dataloaders = {
        'train' : train_loader,
        'valid' : valid_loader
    }

    model, history = train_model(model, criterion, optimizer, scheduler, num_epochs, dataloaders, dataset_sizes, device, fold)
    
    return model, history

In [20]:
fold = 0
valid_df = df_train[df_train.fold == fold]
train_df = df_train[df_train.fold != fold]
    
train_data = ShopeeDataset(TRAIN_DIR, train_df, transforms=data_transforms["train"])
valid_data = ShopeeDataset(TRAIN_DIR, valid_df, transforms=data_transforms["valid"])
    
dataset_sizes = {
        'train' : len(train_data),
        'valid' : len(valid_data)
    }
    
train_loader = DataLoader(dataset=train_data, batch_size=CFG.batch_size, num_workers=0, pin_memory=True, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=CFG.batch_size, num_workers=0, pin_memory=True, shuffle=False)
    
dataloaders = {
        'train' : train_loader,
        'valid' : valid_loader
    }


In [21]:
print(valid_df)

                                             image_path  label  fold
0     C:\Psychopy\Experiment\Cat experiment\cat data...      1     0
1     C:\Psychopy\Experiment\Cat experiment\cat data...      0     0
2     C:\Psychopy\Experiment\Cat experiment\cat data...      1     0
3     C:\Psychopy\Experiment\Cat experiment\cat data...      0     0
4     C:\Psychopy\Experiment\Cat experiment\cat data...      1     0
...                                                 ...    ...   ...
1595  C:\Psychopy\Experiment\Cat experiment\cat data...      0     0
1596  C:\Psychopy\Experiment\Cat experiment\cat data...      1     0
1597  C:\Psychopy\Experiment\Cat experiment\cat data...      0     0
1598  C:\Psychopy\Experiment\Cat experiment\cat data...      1     0
1599  C:\Psychopy\Experiment\Cat experiment\cat data...      0     0

[1600 rows x 3 columns]


In [22]:
print(len(valid_df))

1600


Load Model

In [12]:
model = timm.create_model(CFG.model_name, pretrained=True)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, CFG.embedding_size) # seeting final calssification layer to make prodiction for our label
out = model(torch.randn(1, 3, CFG.img_size, CFG.img_size))
print(f'Embedding shape: {out.shape}')

model.to(CFG.device);

Embedding shape: torch.Size([1, 224])


Approach to change the final activate layre from SiLU to ReLU

In [13]:
model.act2=nn.ReLU()

In [14]:
model.act2

ReLU()

In [ ]:
# for i, (name, layer) in enumerate(model.named_modules()):
#     if isinstance(layer, nn.SiLU):
#         print(name)
# #         print(type(name))
#         print(layer)


In [15]:
model.named_modules

<bound method Module.named_modules of EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

Custom Implementation (Loss function)

In [16]:
class SupervisedContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.1):
        super(SupervisedContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, feature_vectors, labels):
        # Normalize feature vectors
        feature_vectors_normalized = F.normalize(feature_vectors, p=2, dim=1)
        # Compute logits
        logits = torch.div(
            torch.matmul(
                feature_vectors_normalized, torch.transpose(feature_vectors_normalized, 0, 1)
            ),
            self.temperature,
        )
        print('logits:{}'.format(logits))
        print('torch.squeeze(labels):{}'.format(torch.squeeze(labels)))
        return losses.NTXentLoss(temperature=0.07)(logits, torch.squeeze(labels))

In [17]:
criterion = SupervisedContrastiveLoss(temperature=CFG.temperature).to(CFG.device) # Custom Implementation
# criterion = losses.SupConLoss(temperature=CFG.temperature).to(CFG.device)
optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay) #set up optimizer
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr)

Run Fold 0 - Star training

In this case, Fold label means the traning label. For example, when fold label is 0, the data with 0 label will be setted for traning and another will be setted for validation.

In [18]:
model, history = run_fold(model, criterion, optimizer, scheduler, device=CFG.device, fold=1, num_epochs=CFG.num_epochs)

Epoch 1/10
----------


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(False, device='cuda:0')
ouputs_shape:torch.Size([8, 224])
ouputs:tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddmmBackward>)
outputs_len:8
labels:tensor([0, 0, 1, 1, 0, 1, 0, 1], device='cuda:0')
logits:tensor([[nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       dtype=torch.float16, grad_fn=<DivBackward0>)
torch.squeeze(labels

tensor(False, device='cuda:0')
ouputs_shape:torch.Size([8, 224])
ouputs:tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddmmBackward>)
outputs_len:8
labels:tensor([0, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
logits:tensor([[nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       dtype=torch.float16, grad_fn=<DivBackward0>)
torch.squeeze(labels

KeyboardInterrupt: 

Visualize Training & Validation Metrics

In [ ]:
plt.style.use('fivethirtyeight')
plt.rcParams["font.size"] = "20"
fig = plt.figure(figsize=(22,8))
epochs = list(range(CFG.num_epochs))
plt.plot(epochs, history['train loss'], label='train loss')
plt.plot(epochs, history['valid loss'], label='valid loss')
plt.ylabel('Loss', fontsize=20)
plt.xlabel('Epoch', fontsize=20)
plt.legend()
plt.title('Loss Curve');